In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv("data/Matches Data with Rankings.csv")

In [4]:
data = data.dropna()

In [5]:
data["Winner_HoAw"] = data.apply(lambda row: 'Home' if row['winner'] == row['home_team']  else ('Away' if row['winner'] == row['away_team'] else 'Draw'), axis=1)
data.insert(data.columns.get_loc('winner') + 1, 'Winner_HoAw', data.pop('Winner_HoAw'))
data

,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner,Winner_HoAw,year,rank_home_team,rank_away_team
0,Poland,Serbia,1.0,0.0,Friendly,Poznań,Poland,False,Poland,Home,2016,0.505621,-0.939010
1,Ukraine,Serbia,2.0,0.0,Friendly,Kharkiv,Ukraine,False,Ukraine,Home,2016,-0.361158,-0.939010
2,Poland,Netherlands,1.0,2.0,Friendly,Gdańsk,Poland,False,Netherlands,Away,2016,0.505621,0.216695
3,Austria,Netherlands,0.0,2.0,Friendly,Vienna,Austria,False,Netherlands,Away,2016,-0.505621,0.216695
4,England,Netherlands,1.0,2.0,Friendly,London,England,False,Netherlands,Away,2016,0.794547,0.216695
...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,Poland,Ukraine,3.0,1.0,Friendly,Warsaw,Poland,False,Poland,Home,2024,-0.361158,0.072232
521,England,Belgium,2.0,2.0,Friendly,London,England,False,Draw,Draw,2024,1.372399,1.516862
524,Germany,Netherlands,2.0,1.0,Friendly,Frankfurt,Germany,False,Germany,Home,2024,0.505621,1.083473
528,Slovenia,Portugal,2.0,0.0,Friendly,Ljubljana,Slovenia,False,Slovenia,Home,2024,-1.372399,1.227936


In [6]:
df = data[["home_team", "away_team", "rank_home_team", "rank_away_team","Winner_HoAw"]]

In [7]:
"""scaler = LabelEncoder()
df["Result"] = scaler.fit_transform(df[["Winner_HoAw"]])"""

'scaler = LabelEncoder()\ndf["Result"] = scaler.fit_transform(df[["Winner_HoAw"]])'

In [19]:
X = df.drop("Winner_HoAw",axis=1)
y = df["Winner_HoAw"]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=33)

In [23]:
df

,home_team,away_team,rank_home_team,rank_away_team,Winner_HoAw
0,Poland,Serbia,0.505621,-0.939010,Home
1,Ukraine,Serbia,-0.361158,-0.939010,Home
2,Poland,Netherlands,0.505621,0.216695,Away
3,Austria,Netherlands,-0.505621,0.216695,Away
4,England,Netherlands,0.794547,0.216695,Away
...,...,...,...,...,...
517,Poland,Ukraine,-0.361158,0.072232,Home
521,England,Belgium,1.372399,1.516862,Draw
524,Germany,Netherlands,0.505621,1.083473,Home
528,Slovenia,Portugal,-1.372399,1.227936,Home


In [25]:
from pycaret.classification import *

clf = setup(data=df, 
            target = 'Winner_HoAw', 
            session_id=13, 
            remove_multicollinearity = True ,
            multicollinearity_threshold = 0.8,
            feature_selection=True,
            n_features_to_select=4,
            experiment_name='swiss1',
            use_gpu=True,
            n_jobs=-1);

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the prov

,Description,Value
0,Session id,13
1,Target,Winner_HoAw
2,Target type,Multiclass
3,Target mapping,"Away: 0, Draw: 1, Home: 2"
4,Original data shape,"(507, 5)"
5,Transformed data shape,"(507, 5)"
6,Transformed train set shape,"(354, 5)"
7,Transformed test set shape,"(153, 5)"
8,Numeric features,2
9,Categorical features,2


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the prov

In [27]:
best_model = compare_models(fold=5, sort='F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
qda,Quadratic Discriminant Analysis,0.5002,0.0000,0.5002,0.4795,0.4697,0.2224,0.2314,0.1760
nb,Naive Bayes,0.4944,0.0000,0.4944,0.4805,0.4658,0.2142,0.2241,0.1560
ada,Ada Boost Classifier,0.4749,0.0000,0.4749,0.4538,0.4558,0.1797,0.1829,0.2280
lda,Linear Discriminant Analysis,0.5002,0.0000,0.5002,0.4784,0.4532,0.2054,0.2190,0.1700
lr,Logistic Regression,0.4974,0.0000,0.4974,0.4673,0.4445,0.1993,0.2143,0.1680
ridge,Ridge Classifier,0.4917,0.0000,0.4917,0.4925,0.4352,0.1885,0.2051,0.1740
gbc,Gradient Boosting Classifier,0.4465,0.0000,0.4465,0.4323,0.4352,0.1457,0.1469,0.4040
lightgbm,Light Gradient Boosting Machine,0.4408,0.0000,0.4408,0.4354,0.4321,0.1367,0.1376,0.2420
svm,SVM - Linear Kernel,0.4548,0.0000,0.4548,0.4850,0.4288,0.1572,0.1794,0.1620
xgboost,Extreme Gradient Boosting,0.4212,0.0000,0.4212,0.4210,0.4168,0.1161,0.1169,0.7760


In [29]:
qda = create_model('qda', fold=5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4789,0.0000,0.4789,0.4503,0.4449,0.1927,0.2019
1,0.4789,0.0000,0.4789,0.4573,0.4433,0.1939,0.2046
2,0.4507,0.0000,0.4507,0.4225,0.4186,0.1483,0.1549
3,0.5211,0.0000,0.5211,0.4871,0.4901,0.2517,0.2595
4,0.5714,0.0000,0.5714,0.5805,0.5515,0.3254,0.3360
Mean,0.5002,0.0000,0.5002,0.4795,0.4697,0.2224,0.2314
Std,0.0421,0.0000,0.0421,0.0545,0.0470,0.0611,0.0619


In [31]:
custom_grid_qda = {
    'reg_param': [0.0, 0.01, 0.1, 0.2, 0.5, 0.7, 1.0]
}


tuned_qda = tune_model(qda, 
                       fold=10,               
                       n_iter=200,             
                       optimize='F1', 
                       search_library='scikit-learn',
                       choose_better=True,
                       custom_grid=custom_grid_qda)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5278,0.0000,0.5278,0.5010,0.4948,0.2697,0.2832
1,0.5000,0.0000,0.5000,0.4865,0.4582,0.2385,0.2599
2,0.4722,0.0000,0.4722,0.4440,0.4478,0.1719,0.1767
3,0.4444,0.0000,0.4444,0.3553,0.3802,0.1529,0.1707
4,0.4286,0.0000,0.4286,0.4354,0.4029,0.1304,0.1407
5,0.4571,0.0000,0.4571,0.4156,0.4218,0.1419,0.1483
6,0.5429,0.0000,0.5429,0.5705,0.5143,0.2821,0.2970
7,0.5429,0.0000,0.5429,0.5102,0.5218,0.2866,0.2901
8,0.5714,0.0000,0.5714,0.5619,0.5193,0.3312,0.3546


Fitting 10 folds for each of 7 candidates, totalling 70 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [34]:
prediction_data = pd.read_csv("data/Achtel_finals_updated_withPWERrank.csv")

In [63]:
match_list = prediction_data.drop("Group",axis=1)
match_list.rename(columns={'Home_Team': 'home_team', 'Away_Team': 'away_team', 'Home_Team_Rank': 'rank_home_team', 'Away_Team_Rank': 'rank_away_team'},inplace=True)

In [38]:
predictions = predict_model(tuned_qda, data=prediction_data)

In [156]:
match_list

,home_team,away_team,rank_home_team,rank_away_team
0,Germany,Scotland,0.505621,-0.794547
1,Germany,Hungary,0.505621,-0.216695
2,Scotland,Hungary,-0.794547,-0.216695
3,Hungary,Switzerland,-0.216695,0.361158
4,Scotland,Switzerland,-0.794547,0.361158
5,Switzerland,Germany,0.361158,0.505621
6,Spain,Croatia,0.939010,0.650084
7,Spain,Italy,0.939010,0.794547
8,Croatia,Italy,0.650084,0.794547
9,Italy,Albania,0.794547,-1.516862


In [107]:
match_list

,home_team,away_team,rank_home_team,rank_away_team
0,Germany,Scotland,0.505621,-0.794547
1,Germany,Hungary,0.505621,-0.216695
2,Scotland,Hungary,-0.794547,-0.216695
3,Hungary,Switzerland,-0.216695,0.361158
4,Scotland,Switzerland,-0.794547,0.361158
5,Switzerland,Germany,0.361158,0.505621
6,Spain,Croatia,0.939010,0.650084
7,Spain,Italy,0.939010,0.794547
8,Croatia,Italy,0.650084,0.794547
9,Italy,Albania,0.794547,-1.516862


In [8]:
prediction = pd.read_csv("prediction_QDA")

In [40]:
predictions

,home_team,away_team,rank_home_team,rank_away_team,prediction_label,prediction_score
0,Portugal,Switzerland,1.227936,0.361158,Home,0.5524
1,Netherlands,Ukraine,1.083473,0.072232,Home,0.5838
2,Spain,Turkey,0.939010,-0.939010,Home,0.7618
3,Germany,Denmark,0.505621,0.216695,Draw,0.9188
4,Belgium,Croatia,1.516862,0.650084,Home,0.5531
5,France,Czech Republic,1.661325,-0.650084,Home,0.8291
6,England,Austria,1.372399,-0.072232,Home,0.6750
7,Hungary,Italy,-0.216695,0.794547,Away,0.6032
